In [1]:
from sentence_transformers import SentenceTransformer, util
from LexRank import degree_centrality_scores
from tqdm import tqdm
import nltk
import pandas as pd
import numpy as np
import torch
import pickle

In [2]:
model_path='/media/marcin/Dane/model/'
# model_path='model/'

In [ ]:
import psutil
psutil.virtual_memory()

Error: Session cannot generate requests

In [3]:
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/polish.pickle')

[nltk_data] Downloading package punkt to /home/marcin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
df = pd.read_csv('parsed/corpus/all.csv', index_col=0, na_filter=False)
docs = df.text.to_list()

In [5]:
model = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')

In [6]:
corpus2015 = df[df.rok == '2015']
docs2015 = corpus2015.text.to_list()

In [7]:
def embed_doc(doc):
    # Split the document into sentences
    sentences = tokenizer.tokenize(doc)

    # Compute the sentence embeddings
    embeddings = model.encode(sentences, convert_to_tensor=True)

    # Compute the pair-wise cosine similarities
    cos_scores = util.pytorch_cos_sim(embeddings, embeddings).numpy()

    # Compute the centrality for each sentence
    centrality_scores = degree_centrality_scores(cos_scores, threshold=None)

    # We argsort so that the first element is the sentence with the highest score
    most_central_sentence_indices = np.argsort(-centrality_scores)

    # Return mean embedding of top 5 sentences
    best_embeddings = [e for i, e in enumerate(embeddings) if i in most_central_sentence_indices[:5]]
    
    mean_embedding = torch.mean(torch.stack(best_embeddings), dim=0)

    return mean_embedding

embeddings2015 = [embed_doc(doc) for doc in tqdm(docs2015)]

  0%|          | 0/41930 [00:00<?, ?it/s]

In [ ]:
with open(f'{model_path}embeddings-lexrank-2015.pkl', "wb") as fOut:
    pickle.dump(embeddings2015, fOut, protocol=pickle.HIGHEST_PROTOCOL)

In [81]:
embeddings.shape

torch.Size([299, 768])

In [83]:
best_embeddings = [e for i, e in enumerate(embeddings) if i in most_central_sentence_indices[:5]]

In [101]:
embeddings.shape

torch.Size([299, 768])

In [102]:
mean_embedding = torch.mean(torch.stack(best_embeddings), dim=0)

In [82]:
#Print the 5 sentences with the highest scores
print("Summary:")
for idx in most_central_sentence_indices[0:20]:
    print(centrality_scores[idx], sentences[idx].strip())

Summary:
1.4220720242853406 W związku z tym, proszę państwa, że zależy nam na określonym uczniu, tak jak i państwu, tak jak rodzicom, uczeń ma posiadać kilka kompetencji tych najbardziej podstawowych.
1.4045924258226286 Proszę państwa, mówimy o 4-letnim liceum, bo taka jest konieczność, bo tak tylko możemy mówić o wykształceniu naszych obywateli, dlatego że ostatnia zmiana podstawy programowej - wprowadzana właściwie z miesiąca na miesiąc, proszę sobie przypomnieć daty - doprowadziła do tego, że mamy 4-letnie gimnazjum, kończymy podstawę programową gimnazjum w I klasie, a liceum jest półtorarocznym kursem przygotowującym do matury.
1.4039751338627113 Dlatego, proszę państwa, mówimy o zmniejszeniu obwodów szkół.
1.3734664624078194 Jednocześnie, proszę państwa, mówimy o czytaniu, o tym, że rzeczywiście będziemy to czytanie wzmacniać różnymi projektami, na które są oczywiście pieniądze, ale również poprzez podstawy programowe i czytanie dzieciom w szkole.
1.3681254541205499 Tam oczywiście

In [47]:
indexes_with_topics = [i for i, t in  enumerate(topics) if t != -1]

In [ ]:
indexes_with_topics

In [59]:
df['text_lengts'] = [len(x.split()) for x in df['text']]

In [66]:
df_with_topics = df[df.topic != -1]

In [76]:
df[df.id.str.endswith('00031-01-div-56')].squeeze().text

'Panie Marszałku! Wysoka Izbo! Rodzice kochają swoje dzieci i płacą podatki, dlatego pragną dobrej szkoły. Szkoły, która zapewnia doskonałą jakość edukacji, która nie tylko testuje, ale wychowuje, która gwarantuje bezpieczeństwo i która daje satysfakcję nauczycielowi i radość z chodzenia do szkoły właśnie dzieciom. Tak zostało zaplanowane Prawo oświatowe, tak zostały zaplanowane przepisy wprowadzające i tak skonstruowane są systematyczne plany na przyszłość, na całą kadencję. Chcemy, proszę państwa, podnieść jakość kształcenia. Rzeczywiście bardzo często w przestrzeni medialnej słyszycie państwo o międzynarodowych badaniach, które pokazują, że piętnastolatek dobrze odtwarza. Powinniśmy być dumni z tego faktu. Rzeczywiście odtwarza matematykę, czyta, ma umiejętności przyrodnicze. Niestety tylko odtwarza. Te same międzynarodowe badania mówią jednoznacznie, że nie potrafi rozwiązywać problemów, nie współpracuje w grupie, nie potrafi zastosować prostego wzoru matematycznego, że zwyczajnie 

In [69]:
df[df.topic != -1].sort_values(by=['text_lengts'],ascending=False)

,id,speaker,date,text,posel,klub,lista,okreg,text_lengts,topic
1,PPC-201519-sjm-ppxxx-00035-02-div-4,Witold Waszczykowski,2017-02-09,Panie Prezydencie! Panie Marszałku! Szanowna P...,Witold Waszczykowski,Klub Parlamentarny Prawo i Sprawiedliwość,Prawo i Sprawiedliwość,Sieradz,7641,195
12,PPC-201519-sjm-ppxxx-00039-03-div-26,Beata Szydło,2017-04-07,Panie Marszałku! Wysoka Izbo! Można powiedzieć...,Beata Szydło,Klub Parlamentarny Prawo i Sprawiedliwość,Prawo i Sprawiedliwość,Kraków,6750,195
27,PPC-201519-sjm-ppxxx-00031-01-div-56,Anna Zalewska,2016-11-29,Panie Marszałku! Wysoka Izbo! Rodzice kochają ...,Anna Zalewska,Klub Parlamentarny Prawo i Sprawiedliwość,Prawo i Sprawiedliwość,Wałbrzych,5241,94
34,PPC-201519-sjm-ppxxx-00032-01-div-69,Jacek Sasin,2016-12-13,Pani Marszałek! Wysoka Izbo! W dniu 30 wrześni...,Jacek Sasin,Klub Parlamentarny Prawo i Sprawiedliwość,Prawo i Sprawiedliwość,Warszawa,4528,229
3,PPC-201519-sjm-ppxxx-00035-02-div-8,Grzegorz Schetyna,2017-02-09,Panie Marszałku! Pani Premier! Panie Premierze...,Grzegorz Schetyna,Klub Parlamentarny Platforma Obywatelska - Koa...,Platforma Obywatelska,Kielce,4452,195
...,...,...,...,...,...,...,...,...,...,...
56,PPC-201519-sjm-ppxxx-00064-02-div-116,Dariusz Piontkowski,2018-06-15,Panie Marszałku! Wysoka Izbo! Komisja ochrony ...,Dariusz Piontkowski,Klub Parlamentarny Prawo i Sprawiedliwość,Prawo i Sprawiedliwość,Białystok,27,240
1,PPC-201519-sjm-ppxxx-00008-03-div-4,Andrzej Matusiewicz,2016-01-15,Panie Marszałku! Wysoka Izbo! Komisja Nadzwycz...,Andrzej Matusiewicz,Klub Parlamentarny Prawo i Sprawiedliwość,Prawo i Sprawiedliwość,Krosno,27,240
167,PPC-201519-sjm-ppxxx-00031-01-div-332,Stefan Romecki,2016-11-29,Pani Marszałek! Panie Ministrze! Wysoka Izbo! ...,Stefan Romecki,Klub Poselski Kukiz'15,Kukiz'15,Koszalin,26,173
32,PPC-201519-sjm-ppxxx-00044-03-div-70,Artur Soboń,2017-06-22,Panie Marszałku! Wysoka Izbo! W czasie drugieg...,Artur Soboń,Klub Parlamentarny Prawo i Sprawiedliwość,Prawo i Sprawiedliwość,Lublin,26,240


In [53]:
topics[585]

103

In [54]:
df.iloc[585].text

'Panie Marszałku! Panie Ministrze! Wysoka Izbo! Na rozwój handlu zagranicznego w Polsce wpływają różne czynniki, przede wszystkim polityka władz państwowych, które wyznaczają kierunek wymiany handlowej, popyt na określone dobra czy usługi oferowane na rynkach, zapotrzebowanie gospodarki Polski na towary wymagane do rozwoju poszczególnych sektorów. To od władz Polski zależy, w jakim kierunku będzie zmierzać handel zagraniczny. Zagraniczna wymiana handlowa to bardzo często wyznacznik dobrobytu państwa. Często stanowi impuls do rozwoju gospodarczego. Dodatnie saldo bilansu handlowego jest bardzo korzystne z punktu widzenia gospodarki. Pozwala utrzymać miejsca pracy i wpływa korzystnie na rozwój krajowej gospodarki. Panie ministrze, według danych Ministerstwa Rozwoju ujemne saldo polsko-chińskich obrotów handlowych w 2015 r. uległo zwiększeniu. Dlaczego mamy tak duży deficyt w handlu z Chinami? Czy mamy szansę to przełamać? Czy wizyta prezydenta Andrzeja Dudy w Chinach będzie miała wpływ n